In [1]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [2]:
# Creates the antecedents, or inputs, to decide the most efficient solution.
risk = ctrl.Antecedent(np.arange(0, 101, 1), 'risk')            # Risk of the generated action
time = ctrl.Antecedent(np.arange(0, 101, 1), 'time')            # Time to execute the action
energy = ctrl.Consequent(np.arange(0, 101, 1), 'energy')        # Considers the current energy of the system.
complexity = ctrl.Consequent(np.arange(0, 101, 1), 'complexity')# 
impact = ctrl.Consequent(np.arange(0, 101, 1), 'impact')        # How much it would affect the mission

# Output variable to decide the best course of action, the fuzzy set of each action depends on the State (S) of the system
action = ctrl.Consequent(np.arange(0, 11, 1), 'action')

In [3]:
# Fuzzy sets for input variables, the sets are splitted in 3 different categories which are (low, medium, high)
risk.automf(3, variable_type='quantity')
time.automf(3, 'quantity', invert=True)
energy.automf(3, 'quantity', invert=True)
complexity.automf(3, 'quantity')
impact.automf(3, 'quantity')

In [4]:
# Create the possible actions for each state S
def actions(state):
    action.terms = {} # Clear the terms
    if state == "S1": # Normal
        action['remain_normal'] = fuzz.trapmf(action.universe, [0, 0, 8, 10])
        action['emergency_landing'] = fuzz.trapmf(action.universe, [9, 9, 10, 10])
        return 
    elif state == "S2": # GPS Spoofing
        action['restore_attack'] = fuzz.trimf(action.universe, [0, 0, 3])
        action['change_route'] = fuzz.trimf(action.universe, [1, 3, 5])
        action['activate_redundancy'] = fuzz.trimf(action.universe, [3, 5, 7])
        action['use_occupancy_maps'] = fuzz.trimf(action.universe, [5, 7, 9])
        action['emergency_landing'] = fuzz.trimf(action.universe, [7, 10, 10])
        return
    elif state == "S3": # Jamming GPS
        action['change_altitude'] = fuzz.trimf(action.universe, [0, 0, 3])
        action['restore_system'] = fuzz.trapmf(action.universe, [1, 4, 5, 7])
        action['activate_redundancy'] = fuzz.trapmf(action.universe, [3, 6, 7, 9])
        action['emergency_landing'] = fuzz.trimf(action.universe, [7, 10, 10])
        return
    elif state == "S4": # Engine
        action['activate_redundancy'] = fuzz.trapmf(action.universe, [0, 0, 4, 6])
        action['emergency_landing'] = fuzz.trapmf(action.universe, [4 ,7, 10, 10])
    elif state == "S5": # Aileron or elevator
        action['change_altitude'] = fuzz.trapmf(action.universe, [0, 0, 4, 6])
        action['emergency_landing'] = fuzz.trapmf(action.universe, [4 ,7, 10, 10])
        return
    elif state == "S6": # Rudder
        action['navigation_adjustment'] = fuzz.trapmf(action.universe, [0, 0, 4, 6])
        action['emergency_landing'] = fuzz.trapmf(action.universe, [4 ,7, 10, 10])
        return

In [33]:
# Create the rules for the fuzzy system
def rules(state):
    if state == "S1": # Normal
        return [
            ctrl.Rule((risk['high'] | complexity['high']) & energy['low'], action['emergency_landing'])
        ]
    elif state == "S2": # GPS Spoofing
        return [
            ctrl.Rule(risk['low'] & impact['low'] & complexity['low'], action['restore_attack']),
            ctrl.Rule(time['high'] & energy['average'] & impact['average'], action['change_route']),
            ctrl.Rule(time['average'] & energy['average'] & impact['average'], action['activate_redundancy']),
            ctrl.Rule(time['low'] & energy['average'] & impact['high'], action['use_occupancy_maps']),
            ctrl.Rule((risk['high'] | complexity['high']) & energy['low'], action['emergency_landing'])
        ]
    elif state == "S3": # Jamming GPS
        return [
            ctrl.Rule(risk['low'] & impact['low'] & complexity['low'], action['change_altitude']),
            ctrl.Rule(time['high'] & energy['average'] & impact['average'], action['restore_system']),
            ctrl.Rule(complexity['average'] & energy['average'] & time['average'], action['activate_redundancy']),
            ctrl.Rule((risk['high'] | complexity['high']) & energy['low'], action['emergency_landing'])
        ]
    elif state == "S4": # Engine
        return [
            ctrl.Rule((risk['average'] | complexity['average']) & energy['high'], action['activate_redundancy']),
            ctrl.Rule((risk['high'] | complexity['high']) & energy['low'], action['emergency_landing'])
        ] 
    elif state == "S5": # Aileron
        return [
            ctrl.Rule((risk['average'] | complexity['average']) & energy['high'], action['change_altitude']),
            ctrl.Rule((risk['high'] | complexity['high']) & energy['low'], action['emergency_landing'])
        ] 
    elif state == "S6": # Rudder
        return [
            ctrl.Rule((risk['average'] | complexity['average']) & energy['high'], action['navigation_adjustment']),
            ctrl.Rule((risk['high'] | complexity['high']) & energy['low'], action['emergency_landing'])
        ] 


In [34]:
def decision_name(state, action_value):
    if state == "S1":
        if action_value < 5:
            return "Remain Normal"
        else:
            return "Emergency Landing"
    elif state == "S2": 
        if action_value < 2:
            return "Restore Attack"
        elif action_value < 4:
            return "Change Route"
        elif action_value < 6:
            return "Activate Redundancy"
        elif action_value < 8:
            return "Use Occupancy Maps"
        else:
            return "Emergency Landing"
    elif state == "S3":
        if action_value < 2:
            return "Change Altitude"
        elif action_value < 5:
            return "Restore System"
        elif action_value < 8:
            return "Activate Redundancy"
        else:
            return "Emergency Landing"
    elif state == "S4":
        if action_value < 5:
            return "Activate Redundancy"
        else:
            return "Emergency Landing"
    elif state == "S5":
        if action_value < 5:
            return "Change Altitude"
        else:
            return "Emergency Landing"
    elif state == "S6":
        if action_value < 5:
            return "Navigation Adjustment"
        else:
            return "Emergency Landing"
        

In [35]:
# Function to get the appropriate action based on the current state and inputs
def get_action(state, risk_value, time_value, energy_value, complexity_value, impact_value):
    actions(state)
    rule_set = rules(state)
    action_ctrl = ctrl.ControlSystem(rule_set)
    action_simulation = ctrl.ControlSystemSimulation(action_ctrl)

    action_simulation.input['risk'] = risk_value
    action_simulation.input['time'] = time_value
    action_simulation.input['energy'] = energy_value
    action_simulation.input['complexity'] = complexity_value
    action_simulation.input['impact'] = impact_value
    
    action_simulation.compute()
    
    action_value = action_simulation.output['action']
    
    return decision_name(state, action_value)

In [32]:
print(get_action("S3", 50, 50, 50, 49, 50))

ValueError: Unexpected input: time